<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/news-headline-generation/news-headline-generation-with-bloom-6b4-clp-german.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Headline Generation with bloom-6b4-clp-german
Evaluating bloom-6b4-clp-german on the task of generating headlines from journalistic text in german.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Suffix `Headline:` is added to prompt the model.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = df['fulltext'] + " Headline:"
df.head()

Generating train split:   0%|          | 0/9245 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1028 [00:00<?, ? examples/s]

Dataset gnad10 downloaded and prepared to /root/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,Wien – Rapid muss wohl bis Saisonende auf Off...
1,"Erfundene Bilder zu Filmen, die als verloren g...",Wien – Die Filmgeschichte ist ein Friedhof de...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,Die Atmosphäre im Silicon Valley ist rau. Da ...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",Moskau – Die russischen Ölproduzenten wollen ...
4,Estland sieht den künftigen österreichischen P...,Wien/Tallinn/Pskow – Die Eltern des künftigen...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("malteos/bloom-6b4-clp-german", device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("malteos/bloom-6b4-clp-german")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-1c5cjdjal7odi --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=

Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
template = {
      "min_new_tokens": 5,
      "max_new_tokens": 20,
      "length_penalty": 1.0,
      "no_repeat_ngram_size": 0,
      "num_beams": 1,
      "repetition_penalty": 1.0,
      "diversity_penalty": 0.0,
      "num_beam_groups": 1,
      "do_sample": False,
      "top_k": 50,
      "top_p": 0.9,
      "temperature": 1.0,
      "early_stopping": True,
      "pad_token_id": 0,
      "eos_token_id": 186,
      "num_return_sequences": 1,
      "return_full_text": False,
}

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                **template)

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
  print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1359: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[{'generated_text': ' Rapid muss wohl bis Saisonende auf Offensivspieler Thomas Murg verzichten. Der im Winter aus Ried ge'}]
[{'generated_text': ' The Forbidden Room (2015) von Guy Maddin, Evan Johnson. Mit Roy Du'}]
[{'generated_text': ' Google-Chef Pichai: Alle Arschlöcher sind jetzt weg.\n'}]
[{'generated_text': ' Russland will Ölförderung einfrieren\n'}]
[{'generated_text': 'Die Geschichte der Firma\n'}]
[{'generated_text': ' Trodat Trotec Holding GmbH\n'}]
[{'generated_text': ' Heracles Almelo hat sich in den Niederlanden erstmals für einen europäischen Bewerb qualifiziert. Dank eines'}]
[{'generated_text': ' ORF-Stiftungsratssitzung: Wrabetz will Zeitplan und Budget einhalten.Die'}]
[{'generated_text': ' Pakistan: 43 killed in airstrike in North Waziristan\n'}]
[{'generated_text': ' Bundesliga: Hamburger SV – Bayern München (20.30 Uhr)\n'}]
[{'generated_text': ' Der FC Bayern München hat am Sonntag in der deutschen Bundesliga einen wichtigen Sieg gefeiert. Die'}]
[{'generated_te

KeyboardInterrupt: ignored